In [86]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


In [87]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print(use_cuda)

False


In [94]:
pd.set_option('display.max_rows', None)
train_csv = pd.read_csv("train.csv")
test_csv = pd.read_csv("test.csv")
samples_submission_csv = pd.read_csv("sample_submission.csv")

In [113]:
# creation label based on train set
#print(train_csv.dtypes)
train_label1 = train_csv['Id']
print(type(train_label1))
train_label2 = train_csv['SalePrice']
print(type(train_label2))
train_label = pd.DataFrame(columns = ['Id', 'SalePrice'])
train_label['Id'] = train_label1
train_label['SalePrice'] = train_label2
print(train_label)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
        Id  SalePrice
0        1     208500
1        2     181500
2        3     223500
3        4     140000
4        5     250000
5        6     143000
6        7     307000
7        8     200000
8        9     129900
9       10     118000
10      11     129500
11      12     345000
12      13     144000
13      14     279500
14      15     157000
15      16     132000
16      17     149000
17      18      90000
18      19     159000
19      20     139000
20      21     325300
21      22     139400
22      23     230000
23      24     129900
24      25     154000
25      26     256300
26      27     134800
27      28     306000
28      29     207500
29      30      68500
30      31      40000
31      32     149350
32      33     179900
33      34     165500
34      35     277500
35      36     309000
36      37     145000
37      38     153000
38      39     109000
39      40      82000
40      41     160000
41  

In [115]:
# Trainin set
print(train_csv.shape)
train_wo_SP = train_csv.drop('SalePrice', axis='columns')
print(train_wo_SP.shape)



(1460, 81)
(1460, 80)


In [116]:
all_features = pd.concat([train_wo_SP, test_csv], keys=["train", "test"])

all_features_dummies = pd.get_dummies(all_features)

#print(all_features_dummies.shape)
print(all_features_dummies.loc['test'])

        Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
0     1461          20         80.0    11622            5            6   
1     1462          20         81.0    14267            6            6   
2     1463          60         74.0    13830            5            5   
3     1464          60         78.0     9978            6            6   
4     1465         120         43.0     5005            8            5   
5     1466          60         75.0    10000            6            5   
6     1467          20          NaN     7980            6            7   
7     1468          60         63.0     8402            6            5   
8     1469          20         85.0    10176            7            5   
9     1470          20         70.0     8400            4            5   
10    1471         120         26.0     5858            7            5   
11    1472         160         21.0     1680            6            5   
12    1473         160         21.0   

In [82]:
#numeric_features = train_csv.dtypes[train_csv.dtypes != 'object'].index
#numeric_features

In [83]:
#train_csv.dtypes

In [84]:
#train_csv.describe(include='all').T

In [85]:

#train_csv['SalePrice'].hist()

In [86]:
#sum(train_csv.isna().sum())

In [87]:
#sns.distplot(train_csv['SalePrice'])

In [88]:
#cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars']
#sns.pairplot(train_csv[cols], size = 4);

In [89]:
'''total = train_csv.isnull().sum().sort_values(ascending=False)
percent = (train_csv.isnull().sum()/train_csv.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)'''

"total = train_csv.isnull().sum().sort_values(ascending=False)\npercent = (train_csv.isnull().sum()/train_csv.isnull().count()).sort_values(ascending=False)\nmissing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])\nmissing_data.head(20)"

In [90]:
'''
# data preprocessing

x = train_csv['GrLivArea']
y = train_csv['SalePrice']

x = (x - x.mean()) / x.std()
x = np.c_[np.ones(x.shape[0]), x] 

print(x.shape)
'''

"\n# data preprocessing\n\nx = train_csv['GrLivArea']\ny = train_csv['SalePrice']\n\nx = (x - x.mean()) / x.std()\nx = np.c_[np.ones(x.shape[0]), x] \n\nprint(x.shape)\n"

In [93]:
'''pd.set_option('display.max_rows', None)
train_dummies_in = pd.get_dummies(train_dummies, dummy_na = True)
print(train_dummies_in.shape)
print(train_dummies_in.dtypes)
'''

"pd.set_option('display.max_rows', None)\ntrain_dummies_in = pd.get_dummies(train_dummies, dummy_na = True)\nprint(train_dummies_in.shape)\nprint(train_dummies_in.dtypes)\n"

In [95]:
'''test_dummies = pd.get_dummies(test_csv, dummy_na = True)
test_dummies.shape
'''

'test_dummies = pd.get_dummies(test_csv, dummy_na = True)\ntest_dummies.shape\n'

'Model creation

In [9]:
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        
        # linear layer (330 -> 500)
        self.fc1 = nn.Linear(330, 500)
        
        # linear layer (500 -> 250)
        self.fc2 = nn.Linear(500, 250)
        
        # linear layer (250 -> 125)
        self.fc3 = nn.Linear(250, 125)
        
        # linear layer (125 -> 1)
        self.fc4 = nn.Linear(125, 1)
        
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.175)
        
        # LogSoftmax
        self.LSM = nn.LogSoftmax()
    
    def forward(self, x):
        
        # add 1st hidden layer, with relu activation function
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        #h2
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        #h3
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        #h4
        x = self.fc4(x)
        x = self.LSM(x)

        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_HR = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_patho.cuda()

In [10]:
model_HR

Net(
  (fc1): Linear(in_features=330, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=250, bias=True)
  (fc3): Linear(in_features=250, out_features=125, bias=True)
  (fc4): Linear(in_features=125, out_features=1, bias=True)
  (dropout): Dropout(p=0.175, inplace=False)
  (LSM): LogSoftmax()
)